<a href="https://colab.research.google.com/github/ingenium70/happyDL/blob/master/Cifar_10_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q torch torchvision
import torch
print(torch.__version__)

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import time
#from mpl_toolkits import mplot3d

from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm

#Data preparation

In [0]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainset, valset = torch.utils.data.random_split(trainset, [40000, 10000])
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

partition ={'train': trainset, 'val': valset, 'test': testset}

#Data check

In [0]:
# functions to show an image

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('     %5s' % classes[labels[j]] for j in range(4)))

#Model Architecture 

In [0]:
class MLP(nn.Module):
    def __init__(self, in_dim, out_dim, hid_dim, n_layer, act, dropout, use_bn, use_xavier):
        super(MLP, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hid_dim = hid_dim
        self.n_layer = n_layer
        self.act = act
        self.dropout = dropout
        self.use_bn = use_bn
        self.use_xavier = use_xavier
    
        self.fc1 = nn.Linear(self.in_dim, self.hid_dim)
        self.linears = nn.ModuleList()
        self.bns = nn.ModuleList()

        for i in range(self.n_layer-1):
            self.linears.append(nn.Linear(self.hid_dim, self.hid_dim))  
            if self.use_bn:
                self.bns.append(nn.BatchNorm1d(self.hid_dim))

        self.fc2 = nn.Linear(self.hid_dim, self.out_dim)

        if self.act == 'relu':
            self.act = nn.ReLU()
        elif self.act == 'tanh':
            self.act = nn.Tanh()
        elif self.act == 'sigmoid':
            self.act = nn.Sigmoid()
        else:
            raise ValueError('no valid activation function selected!')

        self.dropout = nn.Dropout(self.dropout)
        if self.use_xavier:
            self.xavier_init()
        
    def forward(self, x):
        x = self.act(self.fc1(x))
        for i in range(len(self.linears)):
            x = self.act(self.linears[i](x))
            x = self.bns[i](x)
            x = self.dropout(x)
        x = self.fc2(x)
        return x

    def xavier_init(self):
        for linear in self.linears:
            nn.init.xavier_normal_(linear.weight)
            linear.bias.data.fill_(0.01)

net = MLP(3072, 10, 100, 4, 'relu', 0.1, True, True)

# Train, Validate, Test and Experiment

In [0]:
#=========== Train ==============#

def train(net, partition, optimizer, criterion, args):
    trainloader = torch.utils.data.DataLoader(partition['train'], batch_size=args.train_batch_size, shuffle=True, num_workers=2)

    net.train()
    optimizer.zero_grad()

    total = 0
    correct = 0
    running_loss = 0.0
    train_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.view(-1, 3072)
        inputs = inputs.cuda()
        labels = labels.cuda()

        # forward + backward + optimize
        outputs = net.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total +=labels.size(0)
        correct +=(predicted == labels).sum().item()
    
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    
    return net, train_loss, train_acc

#=========== Validate ==============#

def validate(net, partition, criterion, args):
    valloader = torch.utils.data.DataLoader(partition['val'], batch_size=args.test_batch_size, shuffle=False, num_workers=2)

    net.eval()

    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images = images.view(-1, 3072)
            images = images.cuda()
            labels = labels.cuda()

            outputs = net.forward(images)
            loss = criterion(outputs, labels)

            val_loss +=loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss/len(valloader)
        val_acc = 100*correct /total
    return val_loss, val_acc

#=========== Test ==============#

def test(net, partition, args):
    testloader = torch.utils.data.DataLoader(partition['test'], batch_size=args.test_batch_size, shuffle=False, num_workers=2)

    net.eval()
   
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.view(-1, 3072)
            images = images.cuda()
            labels = labels.cuda()

            outputs = net.forward(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        test_acc = 100*correct/total

    return test_acc

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#Experiement

In [0]:
def experiment(partition, args):
    net = MLP(args.in_dim, args.out_dim, args.hid_dim, args.n_layer, args.act, args.dropout,
              args.use_bn, args.use_xavier)
    net.cuda()

    criterion = nn.CrossEntropyLoss()
    if args.optim == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('no valid optimizer choice!')

    for epoch in range(args.ep):  # loop over the dataset multiple times
        ts = time.time()
        net, train_loss, train_acc = train(net, partition, optimizer, criterion, args)
        val_loss, val_acc = validate(net, partition, criterion, args)
        te = time.time()
        print('Epoch{}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took {: 2.2f} sec'
                .format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
    
    test_acc = test(net, partition, args)

    return train_loss, val_loss, train_acc, val_acc, test_acc

#Parameter

In [0]:
#=============Random Seed Initialization =================#
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

#===============Model Capacity ==================#
args.in_dim = 3072
args.out_dim = 10
args.hid_dim = 100
args.act = 'relu'

#============Regularization ==================#
args.dropout = 0.2
args.use_bn = True
args.l2 = 0.00001
args.use_xavier = True

#=================Optimizer & Training ==========#
args.optim = 'RMSprop'
args.lr = 0.0015
args.ep = 10

args.train_batch_size = 256
args.test_batch_size = 1024

#=================Experiment Variables =============#
name_var1 = 'n_layer'
name_var2 = 'hid_dim'

list_var1 = [3,5,6]
list_var2 = [500, 300, 700]

for var1 in list_var1:
    for var2 in list_var2:
        setattr(args, name_var1, var1)
        setattr(args, name_var2, var2)
        print(args)
        result = experiment(partition, args)

#Reprot Experiment

In [0]:
fig = plt.figure(figsize=(15,5))
list_val_acc = np.reshape(list_val_acc, (3, 3))

# ====== Validation accuracy Fluctuation ====== #
ax1 = fig.add_subplot(1, 2, 1, projection= '3d')
ax1.contour(list_var1, list_var2, list_val_acc, 50, cmap='binary')
ax1.set_xlabel('n_layer')
ax1.set_ylabel('hid_dim')
ax1.set_zlabel('val_acc')

"""
ax1.plot(list_epoch, list_train_loss, label='train_loss')
ax1.plot(list_epoch, list_val_loss, '--', label='val_loss')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')
ax1.grid()
ax1.legend()
ax1.set_title('epoch vs loss')


# ====== Metric Fluctuation ====== #
ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(list_acc_epoch, list_acc, marker='x', label='Accuracy metric')
ax2.set_xlabel('epoch')
ax2.set_ylabel('Acc')
ax2.grid()
ax2.legend()
ax2.set_title('epoch vs Accuracy')
"""
plt.show()